In [91]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pprint
import itertools
import nltk
import chardet
import string
import matplotlib.pyplot as plt
from PIL import Image
from collections import Counter
from nltk.corpus import stopwords 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from wordcloud import WordCloud, STOPWORDS

In [92]:
# import file
df = pd.read_csv('us_gb_ca_merged.csv') 

#split df into 3 countireis
df_us = df[df['country'] == 'US']
df_gb = df[df['country'] == 'GB']
df_ca = df[df['country'] == 'CA']


### Top video categories

In [93]:
df_gb.head()

,Unnamed: 0,video_id,title,publishedAt,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,description,country,title content,tag content,description content,total count title,total count tag
53108,0,J78aPJ3VyNs,i left youtube for a month and this is what ha...,2020-08-11T16:34:06Z,jacksepticeye,24,2020-08-12T00:00:00Z,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353790,2628,40228,i left youtube for a month and this is what ha...,GB,"['i', 'left', 'youtube', 'for', 'a', 'month', ...","['jacksepticeye', 'funny', 'funny meme', 'meme...","['i', 'left', 'youtube', 'for', 'a', 'month', ...",11,30
53109,1,9nidKH8cM38,taxi cab slayer kills 'to know how it feels',2020-08-11T20:00:45Z,Eleanor Neale,27,2020-08-12T00:00:00Z,eleanor|neale|eleanor neale|eleanor neale true...,236830,16423,209,1642,the first 1000 people to click the link will g...,GB,"['taxi', 'cab', 'slayer', 'kills', ""'to"", 'kno...","['eleanor', 'neale', 'eleanor neale', 'eleanor...","['the', 'first', '1000', 'people', 'to', 'clic...",9,43
53110,2,M9Pmf9AB4Mo,apex legends | stories from the outlands – “th...,2020-08-11T17:00:10Z,Apex Legends,20,2020-08-12T00:00:00Z,apex legends|apex legends characters|new apex ...,2381688,146739,2794,16549,"while running her own modding shop, ramya pare...",GB,"['apex', 'legends', '|', 'stories', 'from', 't...","['apex legends', 'apex legends characters', 'n...","['while', 'running', 'her', 'own', 'modding', ...",10,25
53111,3,kgUV1MaD_M8,nines - clout (official video),2020-08-10T18:30:28Z,Nines,24,2020-08-12T00:00:00Z,nines|trapper of the year|crop circle|nines tr...,613785,37567,669,2101,nines - clout (official video)listen to clout ...,GB,"['nines', '-', 'clout', '(official', 'video)']","['nines', 'trapper of the year', 'crop circle'...","['nines', '-', 'clout', '(official', 'video)li...",5,19
53112,4,49Z6Mv4_WCA,i don't know what im doing anymore,2020-08-11T20:24:34Z,CaseyNeistat,22,2020-08-12T00:00:00Z,[none],940036,87113,1860,7052,ssend love to my sponsor; for a super limited ...,GB,"['i', ""don't"", 'know', 'what', 'im', 'doing', ...",['[none]'],"['ssend', 'love', 'to', 'my', 'sponsor;', 'for...",7,1


In [98]:
df_us = df[df['title'] == 'coney']

#### Top category

In [94]:
#df_gb category video count vs top views : 10: music, 24:Entertainment, 20: gaming
df_gb_topcategory = pd.DataFrame(df_gb.groupby('categoryId')['view_count'].sum()).sort_values(by = 'view_count',ascending=False).reset_index()
video_count = pd.DataFrame(df_gb['categoryId'].value_counts()).reset_index().rename(columns={'index' : 'categoryId',
                                                                                             'categoryId':'video_count'})
df_gb_topcategory = df_gb_topcategory.merge(video_count, how = 'inner', on = 'categoryId')

In [95]:
#split categories
df_gb_10 = df_gb[df_gb['categoryId'] == 10]
df_gb_24 = df_gb[df_gb['categoryId'] == 24]
df_gb_20 =  df_gb[df_gb['categoryId'] == 20]
df_gb_25 =  df_gb[df_gb['categoryId'] == 25]

In [96]:
##Extracting hot topics with NLTK
text = df_gb_25['tags'].str.lower().replace('|', ' ').str.cat(sep=' ')


stop_words = set(stopwords.words('english')) 
  
word_tokens = word_tokenize(text) 
    
filtered_sentence = [] 
  
for w in word_tokens: 
    if w not in stop_words: 
        filtered_sentence.append(w) 
        
# Stemming with NLTK
Stem_words = []
ps =PorterStemmer()
for w in filtered_sentence:
    rootWord=ps.stem(w)
    Stem_words.append(rootWord)
    
# Lemmatization with NLTK
filtered_sentence = list(filter(lambda token: token not in string.punctuation, filtered_sentence))
filtered_sentence

# remove unnecessay words
stopwords = ["'s", "’", "..." , "ft." , "2" ,"x" , "1", "n't", "–", "3", "5", "4",
             "2021","2020","trailer", "de", "official", "season", "video", "official", "season", "episode","la", "le", "je",
             "part", "je", "des","world","day", "10","e", "avec", "‘", "à", "music", "none", "new","lil", "like", "songs", "song",
            "thee","love","bad","g","tv", "voice","game", "news","live","watch", "full", "today", "uk" ]
for word in list(filtered_sentence):  # iterating on a copy since removing will mess things up
    if word in stopwords:
        filtered_sentence.remove(word)
        
#wordcloud
word_could_dict=Counter(filtered_sentence)

wordcloud = WordCloud(width = 1000, height = 500, background_color ='black',  stopwords = stopwords,
                min_font_size = 10).generate_from_frequencies(word_could_dict)


plt.figure(figsize=(15,8))
plt.imshow(wordcloud)
plt.axis("off")
# plt.show()
plt.savefig('gb25_tags_wordcloud.png', bbox_inches='tight')
plt.close()

In [97]:
filtered_sentence = pd.DataFrame(filtered_sentence)
filtered_sentence_unique = pd.DataFrame(filtered_sentence.value_counts())
filtered_sentence_unique = filtered_sentence_unique.rename(columns={'0':'count'})
filtered_sentence_unique.head(50)

,0
0,
philip,233
biden,153
johnson,141
election,126
british,106
bbc|bbc,102
harry,86
news|coronavirus,85
prince,81
